In [213]:
'''VoidFinder - Hoyle & Vogeley (2002)'''



################################################################################
#
# If you have control over your python environment, voidfinder can be installed
# as a normal python package via 'python setup.py install', in which case the 
# below import of 'sys' and 'sys.path.insert(0, '/abspath/to/VoidFinder/python'
# is unnecessary.  If you aren't able to install the voidfinder package,
# you can use the sys.path.insert to add it to the list of available packages
# in your python environment.
#
# Alternately, "python setup.py develop" will 'install' some symlinks which
# point back to the current directory and you can run off the same voidfinder
# repository that you're working on as if it was installed
#
################################################################################


#import sys
#sys.path.insert(1, 'local/path/VAST/VoidFinder/vast/voidfinder/')

################################################################################
#
#   IMPORT MODULES
#
################################################################################

from vast.voidfinder import find_voids, filter_galaxies

from vast.voidfinder.multizmask import generate_mask
from vast.voidfinder.preprocessing import file_preprocess
from vast.voidfinder.table_functions import to_vector, to_array

import pickle
import numpy as np

In [1]:
################################################################################
#
#   USER INPUTS
#
################################################################################


# Number of CPUs available for analysis.
# A value of None will use one less than all available CPUs.
num_cpus = 1

#-------------------------------------------------------------------------------
survey_name = 'vollim_dr7_cbp_102709_'

# File header
# Change these directory paths to where your data is stored, and where you want 
# the output to be saved.
in_directory = '/global/homes/s/schampat/Voids/Void_analysis/Data/'
out_directory = '/global/homes/s/schampat/Voids/Voidfinder_Output/Delta_Limit = 3.00/'


# Input file name
galaxies_filename = 'deltatable.fits'  # File format: RA, dec, redshift, comoving distance, absolute magnitude
#-------------------------------------------------------------------------------

#-------------------------------------------------------------------------------
# Survey parameters
# Note: These can be set to None, in which case VoidFinder will use the limits 
# of the galaxy catalog.
min_z = 0
max_z = 0.107

# Cosmology (uncomment and change values to change cosmology)
# Need to also uncomment relevent inputs in function calls below
Omega_M = 0.26
#h = 1

# Uncomment if you do NOT want to use comoving distances
# Need to also uncomment relevent inputs in function calls below
dist_metric = 'comoving'
#-------------------------------------------------------------------------------

#-------------------------------------------------------------------------------
# Uncomment if you do NOT want to remove galaxies with Mr > -20
# Need to also uncomment relevent input in function calls below
#mag_cut = False


# Uncomment if you do NOT want to remove isolated galaxies
# Need to also uncomment relevent input in function calls below
#rm_isolated = False
#-------------------------------------------------------------------------------

In [215]:
################################################################################
#
#   PREPROCESS DATA
#
################################################################################

galaxy_data_table, dist_limits, out1_filename, out2_filename = file_preprocess(galaxies_filename, 
                                                                               in_directory, 
                                                                               out_directory,
                                                                               mag_cut=False,
                                                                               #rm_isolated=rm_isolated,
                                                                               dist_metric=dist_metric,
                                                                               #min_z=min_z, 
                                                                               #max_z=max_z,
                                                                               Omega_M=Omega_M,
                                                                               #h=h,
                                                                               verbose=0)

print("Dist limits: ", dist_limits)

Loading galaxy data table at:  /global/homes/s/schampat/Voids/Void_analysis/Data/deltatable1.fits
Galaxy data table load time:  0.026051759719848633
Calculating Rgal data table column
Finished Rgal calculation time:  0.45371317863464355
Dist limits:  [3716.344  5350.8853]


In [216]:
################################################################################
#
#   GENERATE MASK
#
################################################################################

mask, mask_resolution = generate_mask(galaxy_data_table, 
                                      max_z, 
                                      dist_metric=dist_metric, 
                                      smooth_mask=True,
                                      #h=h,
                                     )


temp_outfile = open(out_directory + survey_name + 'mask.pickle', 'wb')
pickle.dump((mask, mask_resolution), temp_outfile)
temp_outfile.close()

Generating mask


In [217]:
################################################################################
#
#   FILTER GALAXIES
#
################################################################################

temp_infile = open(out_directory + survey_name + 'mask.pickle', 'rb')
mask, mask_resolution = pickle.load(temp_infile)
temp_infile.close()

wall_coords_xyz, field_coords_xyz, hole_grid_shape, coords_min = filter_galaxies(galaxy_data_table,
                                                                                 survey_name,
                                                                                 out_directory,
                                                                                 mag_cut=False,
                                                                                 delta_cut=True,
                                                                                 dist_limits=dist_limits,
                                                                                 #mag_cut_flag=mag_cut,
                                                                                 #rm_isolated_flag=rm_isolated,
                                                                                 #hole_grid_edge_length=5.0,
                                                                                 dist_metric=dist_metric,
                                                                                 #h=h,
                                                                                 delta_limit=3.00,
                                                                                 verbose=0)

del galaxy_data_table


temp_outfile = open(survey_name + "filter_galaxies_output.pickle", 'wb')
pickle.dump((wall_coords_xyz, field_coords_xyz, hole_grid_shape, coords_min), temp_outfile)
temp_outfile.close()

Filter Galaxies Start
    ra       dec      redshift    Delta        Rgal  
--------- ---------- --------- ------------ ---------
2.5950856 0.03995798 2.9162366    0.5633348 4576.8843
2.5950856 0.03995798  2.918943  -0.16900143 4578.8896
2.5950856 0.03995798  2.921651     0.680851  4580.894
2.5950856 0.03995798 2.9243608    1.1129243  4582.898
2.5950856 0.03995798 2.9270725  -0.09884469 4584.9014
2.5950856 0.03995798 2.9297862   0.29953516 4586.9033
2.5950856 0.03995798 2.9325018  -0.59169257  4588.906
2.5950856 0.03995798 2.9352193   0.11125143  4590.907
2.5950856 0.03995798 2.9379385   -0.8568085 4592.9077
2.5950856 0.03995798 2.9406595    0.9868907  4594.908
      ...        ...       ...          ...       ...
2.6371665 0.06261407 3.0678875 -0.123240516 4686.2715
2.6371665 0.06261407 3.0706985  -0.17032763 4688.2437
2.6371665 0.06261407 3.0735114  0.027964989  4690.215
2.6371665 0.06261407 3.0763261   0.11831474  4692.186
2.6371665 0.06261407  3.079143   0.17852738 4694.1562
2.6371

In [218]:
print(wall_coords_xyz.dtype)

float32


In [219]:
################################################################################
#
#   FIND VOIDS
#
################################################################################


temp_infile = open(survey_name + "filter_galaxies_output.pickle", 'rb')
wall_coords_xyz, field_coords_xyz, hole_grid_shape, coords_min = pickle.load(temp_infile)
temp_infile.close()



find_voids(wall_coords_xyz,
           coords_min.astype(np.float64),
           hole_grid_shape,
           survey_name,
           min_dist = dist_limits[0],
           max_dist = dist_limits[1],
           mask=mask, 
           mask_resolution=mask_resolution,
           #save_after=50000,
           #use_start_checkpoint=True,
           #hole_grid_edge_length=5.0,
           #galaxy_map_grid_edge_length=None,
           #hole_center_iter_dist=1.0,
           maximal_spheres_filename=out1_filename,
           void_table_filename=out2_filename,
           potential_voids_filename=out_directory + survey_name + 'potential_voids_list.txt',
           num_cpus=num_cpus,
           batch_size=10000,
           verbose=0,
           print_after=5.0)

Growing holes
Processed 9542 cells of 10006 empty cells 27.83
WORKER EXITING GRACEFULLY 0
Found a total of 538 potential voids.
Time to grow all holes: 32.439929485321045
Starting volume cut
Found  73 holes to cut 0.6931042671203613
Sorting holes based on radius
Combining holes into unique voids
Starting hole combine
Removing duplicate spheres: 5.125999450683594e-05
Find maximal holes: 0.00022864341735839844
Merging holes into voids: 0.0004127025604248047
Combine time: 0.0048029422760009766
Number of unique voids is 25
Saving maximals to /global/homes/s/schampat/Voids/Voidfinder_Output/Delta_Limit = 3.00/deltatable1._comoving_maximal_noMagCut.txt


In [ ]:
'''VoidFinder - Hoyle & Vogeley (2002)'''






